<p style="background-color: #0055A4; font-size: 48px; font-weight: bold; color: white; padding: 20px; text-align: center;">
🧹 Data Cleaning 
</p>


In [8]:
#Import librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math 
from scipy.stats import pearsonr
import scipy.stats as stats
from plotly.subplots import make_subplots
from scipy.stats import shapiro
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore") 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
import plotly.graph_objects as go
import plotly.express as px
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import skew, shapiro
from scipy.stats import boxcox
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
import itertools
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
import shap
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from collections import Counter
from sklearn.feature_selection import SelectKBest
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from prettytable import PrettyTable
import featuretools as ft

In [9]:
#Import data
train_df = pd.read_csv('C:/Users/Mega-Pc/Freelance/data/fraudTrain.csv', delimiter=',', encoding='utf-8',index_col=0)
test_df = pd.read_csv('C:/Users/Mega-Pc/Freelance/data/fraudTest.csv', delimiter=',', encoding='utf-8',index_col=0)



## 🔄 Check for duplicates

In [10]:
#train_df
duplicate_count = train_df.duplicated().sum()
print(f'Number of duplicate records: {duplicate_count}')


Number of duplicate records: 0


In [11]:
#train_df
duplicate_count = test_df.duplicated().sum()
print(f'Number of duplicate records: {duplicate_count}')


Number of duplicate records: 0


## ✨ Data Transformation

## Log Transformation

In [12]:
#train_df + test_df
skewed_cols = ['amt','city_pop']
for col in skewed_cols:
    train_df[col] = np.log1p(train_df[col])  
    test_df[col] = np.log1p(test_df[col])
     




In [13]:
results = []
for var in  skewed_cols:
    skewness = stats.skew(train_df[var])
    kurtosis = stats.kurtosis(train_df[var])
    
    results.append({
        'Feature':  var ,
        'Skewness': skewness,
        'Kurtosis': kurtosis
    })

results_df = pd.DataFrame(results)

display(results_df)

,Feature,Skewness,Kurtosis
0,amt,-0.298852,-0.527247
1,city_pop,0.606094,-0.301258


## Handle outliers by capping them 

In [14]:
numerical_vars_filtered = [
    "amt",
    "zip",
    "lat",
    "long",
    "city_pop",
    "unix_time",
    "merch_lat",
    "merch_long"
]


In [15]:
#train_df
outliers_info = []

for col in numerical_vars_filtered:
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = train_df[(train_df[col] < lower_bound) | (train_df[col] > upper_bound)]
    
    outliers_info.append({
        'Feature': col,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outliers Count': len(outliers),
        'Total Points': len(train_df)
    })

outliers_df = pd.DataFrame(outliers_info)
print(outliers_df)


      Feature   Lower Bound   Upper Bound  Outliers Count  Total Points
0         amt -7.348234e-01  7.532865e+00             818       1296675
1         zip -4.247050e+04  1.407495e+05               0       1296675
2         lat  2.364065e+01  5.292025e+01            4679       1296675
3        long -1.217580e+02 -5.519800e+01           49922       1296675
4    city_pop  1.650397e+00  1.488145e+01            4168       1296675
5   unix_time  1.307799e+09  1.390337e+09               0       1296675
6   merch_lat  2.389818e+01  5.279255e+01            4967       1296675
7  merch_long -1.218880e+02 -5.524608e+01           41994       1296675


In [16]:
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.clip(df[column], lower_bound, upper_bound)
    return df

columns_with_outliers = [
    "amt",
    "lat",
    "long",
    "city_pop",
    "merch_lat",
    "merch_long"
]

for col in columns_with_outliers:
    train_df = handle_outliers(train_df, col)


In [17]:
# results after capping outliers
outliers_info = []

for col in numerical_vars_filtered:
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = train_df[(train_df[col] < lower_bound) | (train_df[col] > upper_bound)]
    
    outliers_info.append({
        'Feature': col,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outliers Count': len(outliers),
        'Total Points': len(train_df)
    })

outliers_df = pd.DataFrame(outliers_info)
print(outliers_df)


      Feature   Lower Bound   Upper Bound  Outliers Count  Total Points
0         amt -7.348234e-01  7.532865e+00               0       1296675
1         zip -4.247050e+04  1.407495e+05               0       1296675
2         lat  2.364065e+01  5.292025e+01               0       1296675
3        long -1.217580e+02 -5.519800e+01               0       1296675
4    city_pop  1.650397e+00  1.488145e+01               0       1296675
5   unix_time  1.307799e+09  1.390337e+09               0       1296675
6   merch_lat  2.389818e+01  5.279255e+01               0       1296675
7  merch_long -1.218880e+02 -5.524608e+01               0       1296675


In [18]:
#test_df
outliers_info = []

for col in numerical_vars_filtered:
    Q1 = test_df[col].quantile(0.25)
    Q3 = test_df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = test_df[(test_df[col] < lower_bound) | (test_df[col] > upper_bound)]
    
    outliers_info.append({
        'Feature': col,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outliers Count': len(outliers),
        'Total Points': len(train_df)
    })

outliers_df = pd.DataFrame(outliers_info)
print(outliers_df)


      Feature   Lower Bound   Upper Bound  Outliers Count  Total Points
0         amt -7.372033e-01  7.531819e+00             402       1296675
1         zip -4.228650e+04  1.405895e+05               0       1296675
2         lat  2.383005e+01  5.273365e+01            1933       1296675
3        long -1.217322e+02 -5.524100e+01           21104       1296675
4    city_pop  1.691879e+00  1.480513e+01            1697       1296675
5   unix_time  1.361271e+09  1.400625e+09               0       1296675
6   merch_lat  2.395701e+01  5.275246e+01            2090       1296675
7  merch_long -1.218659e+02 -5.530390e+01           17926       1296675


In [19]:
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.clip(df[column], lower_bound, upper_bound)
    return df

columns_with_outliers = [
    "amt",
    "lat",
    "long",
    "city_pop",
    "merch_lat",
    "merch_long"
]

for col in columns_with_outliers:
    train_df = handle_outliers(test_df, col)


In [20]:
# results after capping outliers
outliers_info = []

for col in numerical_vars_filtered:
    Q1 = test_df[col].quantile(0.25)
    Q3 = test_df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = test_df[(test_df[col] < lower_bound) | (test_df[col] > upper_bound)]
    
    outliers_info.append({
        'Feature': col,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outliers Count': len(outliers),
        'Total Points': len(train_df)
    })

outliers_df = pd.DataFrame(outliers_info)
print(outliers_df)


      Feature   Lower Bound   Upper Bound  Outliers Count  Total Points
0         amt -7.372033e-01  7.531819e+00               0        555719
1         zip -4.228650e+04  1.405895e+05               0        555719
2         lat  2.383005e+01  5.273365e+01               0        555719
3        long -1.217322e+02 -5.524100e+01               0        555719
4    city_pop  1.691879e+00  1.480513e+01               0        555719
5   unix_time  1.361271e+09  1.400625e+09               0        555719
6   merch_lat  2.395701e+01  5.275246e+01               0        555719
7  merch_long -1.218659e+02 -5.530390e+01               0        555719


## Encoding

In [21]:
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

unique_counts = {}
for col in categorical_features:
    unique_counts[col] = {
        "train_unique": train_df[col].nunique(),
        "test_unique": test_df[col].nunique()
    }


unique_counts


{'trans_date_trans_time': {'train_unique': 544760, 'test_unique': 544760},
 'merchant': {'train_unique': 693, 'test_unique': 693},
 'category': {'train_unique': 14, 'test_unique': 14},
 'first': {'train_unique': 341, 'test_unique': 341},
 'last': {'train_unique': 471, 'test_unique': 471},
 'gender': {'train_unique': 2, 'test_unique': 2},
 'street': {'train_unique': 924, 'test_unique': 924},
 'city': {'train_unique': 849, 'test_unique': 849},
 'state': {'train_unique': 50, 'test_unique': 50},
 'job': {'train_unique': 478, 'test_unique': 478},
 'dob': {'train_unique': 910, 'test_unique': 910},
 'trans_num': {'train_unique': 555719, 'test_unique': 555719}}

In [22]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# -----------------------------
# One-Hot Encoding 
# -----------------------------
one_hot_features = ["category", "gender", "state"]

train_df = pd.get_dummies(train_df, columns=one_hot_features, drop_first=False)
test_df = pd.get_dummies(test_df, columns=one_hot_features, drop_first=False)
#test_df
test_df = test_df.reindex(columns=train_df.columns, fill_value=0)

# -----------------------------
# Label Encoding 
# -----------------------------
label_features = ["merchant", "first", "last", "street", "city", "job", "dob"]

for col in label_features:
    le = LabelEncoder()
    le.fit(pd.concat([train_df[col], test_df[col]], axis=0))
    
    train_df[col] = le.transform(train_df[col])
    test_df[col] = le.transform(test_df[col])

# -----------------------------
# high-cardinality features
# -----------------------------

train_df['trans_date_trans_time'] = pd.to_datetime(train_df['trans_date_trans_time'])
test_df['trans_date_trans_time'] = pd.to_datetime(test_df['trans_date_trans_time'])

for df in [train_df, test_df]:
    df['trans_hour'] = df['trans_date_trans_time'].dt.hour
    df['trans_day'] = df['trans_date_trans_time'].dt.day
    df['trans_month'] = df['trans_date_trans_time'].dt.month
    df['trans_weekday'] = df['trans_date_trans_time'].dt.weekday




## Scaling

In [23]:
from sklearn.preprocessing import MinMaxScaler

numerical_features = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

scaler = MinMaxScaler()

train_df[numerical_features] = scaler.fit_transform(train_df[numerical_features])
test_df[numerical_features] = scaler.transform(test_df[numerical_features])


## Feature Engineering

In [26]:
# extract age

from datetime import datetime
def add_age_feature(df):
    df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
    today_year = datetime.today().year
    df['age'] = today_year - df['dob'].dt.year

    df['age_group'] = pd.cut(
        df['age'],
        bins=[0, 25, 40, 60, 100],
        labels=['0', '1', '2', '3'] # 0: young 1:adult 2: mature 3:senior
    )
    
    return df


train_df = add_age_feature(train_df)
test_df = add_age_feature(test_df)

In [28]:
# transaction amount per city pop and categorization into bins
def add_transaction_features(df):
    
    #Transaction amount per city pop
    df["amt_per_capita"] = df["amt"] / (df["city_pop"] + 1)  
    
    #Transaction amount bins 
    df["amt_category"] = pd.qcut(df["amt"], q=3, labels=["1", "2", "3"]) #1:low 2:medium 3:high
    
    return df

# Apply to both train and test
train_df = add_transaction_features(train_df)
test_df = add_transaction_features(test_df)

train_df[["amt", "amt_per_capita", "amt_category"]].head()


,amt,amt_per_capita,amt_category
0,0.096147,0.052815,1
1,0.400029,0.328408,2
2,0.446164,0.274509,2
3,0.499884,0.300219,2
4,0.108143,0.081246,1


In [29]:
#distance between customer and merchant (3 methods)

def haversine_distance(lat1, lon1, lat2, lon2):
    
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371  
    return c * r

def add_distance_features(df):
    df["haversine_dist"] = haversine_distance(
        df["lat"], df["long"], df["merch_lat"], df["merch_long"]
    )
    
    df["manhattan_dist"] = (
        abs(df["lat"] - df["merch_lat"]) + abs(df["long"] - df["merch_long"])
    )
    
    df["euclidean_dist"] = np.sqrt(
        (df["lat"] - df["merch_lat"])**2 + (df["long"] - df["merch_long"])**2
    )
    
    return df


train_df = add_distance_features(train_df)
test_df = add_distance_features(test_df)





In [30]:
# customer level aggregation features 
def add_customer_agg_features(train_df, test_df):
    customer_stats = train_df.groupby("cc_num").agg(
        avg_transaction_amt=("amt", "mean"),
        std_transaction_amt=("amt", "std"),
        transaction_count=("amt", "count")
    ).reset_index()
    
    train_df = train_df.merge(customer_stats, on="cc_num", how="left")
    
    test_df = test_df.merge(customer_stats, on="cc_num", how="left")
    
    train_df["std_transaction_amt"].fillna(0, inplace=True)
    test_df["std_transaction_amt"].fillna(0, inplace=True)
    
    return train_df, test_df


train_df, test_df = add_customer_agg_features(train_df, test_df)



In [31]:
train_df

,trans_date_trans_time,cc_num,merchant,amt,first,last,street,city,zip,lat,...,age,age_group,amt_per_capita,amt_category,haversine_dist,manhattan_dist,euclidean_dist,avg_transaction_amt,std_transaction_amt,transaction_count
0,2020-06-21 12:14:25,4.589232e-04,319,0.096147,151,115,341,157,0.283305,0.350678,...,55,Mature,0.052815,1,2.022390,0.020412,0.018188,0.399822,0.187530,640
1,2020-06-21 12:14:33,7.156895e-04,591,0.400029,163,457,354,16,0.838654,0.570540,...,55,Mature,0.328408,2,3.689786,0.039251,0.033183,0.372070,0.189823,837
2,2020-06-21 12:14:53,7.207342e-04,611,0.446164,24,249,865,61,0.105945,0.582725,...,55,Mature,0.274509,2,3.254732,0.036420,0.029272,0.448421,0.190908,1073
3,2020-06-21 12:15:15,7.194732e-04,222,0.499884,42,457,320,764,0.319498,0.163981,...,55,Mature,0.300219,2,1.683320,0.019047,0.015139,0.392666,0.181532,663
4,2020-06-21 12:15:17,7.064345e-04,292,0.108143,247,261,548,247,0.490300,0.706585,...,55,Mature,0.081246,1,3.964745,0.050211,0.035657,0.389118,0.184189,891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,6.109391e-06,507,0.454531,235,315,531,443,0.630382,0.576504,...,55,Mature,0.359445,2,2.362405,0.022370,0.021246,0.406393,0.191628,628
555715,2020-12-31 23:59:09,7.124010e-04,264,0.589709,171,424,540,401,0.773423,0.180228,...,55,Mature,0.366367,3,3.085534,0.039097,0.027749,0.342446,0.187052,1105
555716,2020-12-31 23:59:15,1.204176e-03,496,0.553152,18,239,126,104,0.993939,0.773833,...,55,Mature,0.386023,3,2.202653,0.027995,0.019810,0.463466,0.183103,1079
555717,2020-12-31 23:59:24,8.051039e-07,75,0.219774,111,342,663,476,0.835016,0.719476,...,55,Mature,0.191892,1,1.463654,0.018181,0.013164,0.387738,0.192601,883
